In [1]:
# Uvoz potrebnih biblioteka
import pandas as pd

In [2]:
# Učitavanje glavnog dataset-a
full_batting_data = pd.read_csv('../resources/full_batting_data.csv')


In [3]:
# Transformacija imena iz "Ime Prezime" u "Prezime, Ime" u glavnom datasetu
full_batting_data['formatted_name'] = full_batting_data['Name'].apply(lambda x: ', '.join(x.split()[::-1]))

# Lista za sakupljanje podataka o sprintu iz svih fajlova
sprint_data_list = []

# Definisanje godina koje su obuhvaćene
years = range(2015, 2020)  # 2020 je ekskluzivno, dakle 2014 do 2019 uključivo

# Učitavanje podataka o sprintu za svaku godinu i njihova transformacija
for year in years:
    sprint_data = pd.read_csv(f'../resources/sprint_speed_data_{year}.csv')
    # Dodavanje kolone 'year' kako bi se sačuvala informacija o godini
    sprint_data['year'] = year
    sprint_data_list.append(sprint_data)






In [4]:
# Kombinovanje svih podataka o sprintu u jedan DataFrame
combined_sprint_data = pd.concat(sprint_data_list)

# Spajanje na osnovu formatiranog imena i godine
extended_data = pd.merge(full_batting_data, combined_sprint_data, left_on=['formatted_name', 'Season'], right_on=['last_name, first_name', 'year'], how='left')

# Provera rezultata
print(extended_data.head())

    IDfg  Season          Name Team  Age    G   AB   PA    H  1B  ...  \
0  13611    2018  Mookie Betts  BOS   25  136  520  614  180  96  ...   
1  10155    2018    Mike Trout  LAA   26  140  471  608  147  80  ...   
2  11579    2015  Bryce Harper  WSN   22  153  521  654  172  91  ...   
3  10155    2015    Mike Trout  LAA   23  159  575  682  172  93  ...   
4  15640    2017   Aaron Judge  NYY   25  155  542  678  154  75  ...   

   player_id  team_id  team  position   age  competitive_runs  bolts  \
0   605141.0    111.0   BOS        RF  25.0             237.0    6.0   
1   545361.0    108.0   LAA        CF  26.0             185.0   36.0   
2   547180.0    120.0   WSH        RF  22.0             223.0    1.0   
3   545361.0    108.0   LAA        CF  23.0             236.0   73.0   
4   592450.0    147.0   NYY        RF  25.0             209.0    NaN   

   hp_to_1b  sprint_speed    year  
0      4.18          28.1  2018.0  
1      4.25          29.2  2018.0  
2      4.34         

In [5]:
extended_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3312 entries, 0 to 3311
Columns: 332 entries, IDfg to year
dtypes: float64(271), int64(53), object(8)
memory usage: 8.4+ MB


In [6]:
# Prikazivanje uzorka spojenih redova sa ključnim informacijama
print(extended_data[['formatted_name',  'Name', 'Season', 'year']].sample(n=10))


          formatted_name              Name  Season    year
2473      Mikolas, Miles     Miles Mikolas    2019     NaN
544      Hamilton, Billy    Billy Hamilton    2014     NaN
3226       O'Hearn, Ryan      Ryan O'Hearn    2019  2019.0
91        Heyward, Jason     Jason Heyward    2015  2015.0
2688  Aza, De, Alejandro  Alejandro De Aza    2016     NaN
1469        Walker, Neil       Neil Walker    2019  2019.0
83        Ozuna, Marcell     Marcell Ozuna    2017  2017.0
2537     Beckham, Gordon    Gordon Beckham    2016  2016.0
3162        Moore, Tyler       Tyler Moore    2015  2015.0
1934       Wheeler, Zack      Zack Wheeler    2018     NaN


In [7]:
# Drop columns from extended_data that are not needed for analysis formatted_name', 'last_name, first_name', 'year','player_id', 'team_id', 'team'

clean_extended_data = extended_data.drop(columns=['formatted_name', 'last_name, first_name', 'year','player_id', 'team_id', 'team', 'position', 'age'])




In [8]:
# Kolonu 'hp_to_1b' nan vrednosti dopun sa mean vrednostima, zaokruzi na dve decimale
clean_extended_data['hp_to_1b'] = clean_extended_data['hp_to_1b'].fillna(clean_extended_data['hp_to_1b'].mean()).round(2)


# Kolunu 'bolts' nan vrednosti zameni sa nulom 
clean_extended_data['bolts'] = clean_extended_data['bolts'].fillna(0)

# Kolunu 'competitive_runs' nan vrednosti zameni sa mean vrednostima, zaokruzi na ceo broj
clean_extended_data['competitive_runs'] = clean_extended_data['competitive_runs'].fillna(clean_extended_data['competitive_runs'].mean()).round(0)



In [9]:
# Prikazi kolone sa nan vrednostima a da ih ima vise od 0 i sortiraj po broju nan vrednosti
nan_columns = clean_extended_data.columns[clean_extended_data.isna().sum() > 0]
nan_counts = clean_extended_data[nan_columns].isna().sum().sort_values(ascending=False)
print(nan_counts)




xwOBA         3312
xSLG          3312
xBA           3312
UN% (sc)      3312
PO%           3312
              ... 
CU-X (sc)        1
CU-Z (sc)        1
wCU/C (sc)       1
wSI/C (sc)       1
vSI (sc)         1
Length: 136, dtype: int64


In [10]:
# Popunjavanje nedostjucih vrednosti u koloni sprint_speed
# Ako za igraca ne postoji vrednost za sprint_speed za neku sezonu, onda se koristi mean vrednost sprint_speed-a za tog igraca
clean_extended_data['sprint_speed'] = clean_extended_data.groupby('Name')['sprint_speed'].transform(lambda x: x.fillna(x.mean()))

# zaokruzi na na jednu decimalu
clean_extended_data['sprint_speed'] = clean_extended_data['sprint_speed'].round(1)


In [11]:
# Izbaci sve redove koji su iz sezone 2014
clean_extended_data = clean_extended_data[clean_extended_data['Season'] != 2014]

In [12]:
# Izbaci sve igrace koji se pojavljuju samo jednu sezonu

clean_extended_data = clean_extended_data.groupby('Name').filter(lambda x: len(x) > 1)



In [13]:
# Sve ostale nan vrednosti u koloni sprint_speed popuni sa mean vrednostima sprint_speed-a
clean_extended_data['sprint_speed'] = clean_extended_data['sprint_speed'].fillna(clean_extended_data['sprint_speed'].mean())

In [14]:
clean_extended_data.head()


,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR,competitive_runs,bolts,hp_to_1b,sprint_speed
0,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,0.220,0.270,NaN,NaN,NaN,10.2,237.0,6.0,4.18,28.1
1,10155,2018,Mike Trout,LAA,26,140,471,608,147,80,...,0.201,0.261,NaN,NaN,NaN,9.5,185.0,36.0,4.25,29.2
2,11579,2015,Bryce Harper,WSN,22,153,521,654,172,91,...,0.118,0.226,NaN,NaN,NaN,9.3,223.0,1.0,4.34,27.7
3,10155,2015,Mike Trout,LAA,23,159,575,682,172,93,...,0.207,0.282,NaN,NaN,NaN,9.3,236.0,73.0,4.20,29.6
4,15640,2017,Aaron Judge,NYY,25,155,542,678,154,75,...,0.157,0.290,NaN,NaN,NaN,8.7,209.0,0.0,4.50,28.0


In [15]:
# Prikazi kolone sa nan vrednostima a da ih ima vise od 0 i sortiraj po broju nan vrednosti
nan_columns = clean_extended_data.columns[clean_extended_data.isna().sum() > 0]
nan_columns_counts = clean_extended_data[nan_columns].isna().sum().sort_values(ascending=False)

In [16]:
# obrisi sve kolone koje imaju vise od 800 nan vrednosti

clean_extended_data = clean_extended_data.drop(columns=nan_columns_counts[nan_columns_counts > 800].index)

In [17]:
# Prikazi sve kolone koje nisu int ili float
non_numeric_columns = clean_extended_data.select_dtypes(exclude=['int', 'float']).columns
non_numeric_columns

Index(['Name', 'Team', 'Dol', 'Age Rng'], dtype='object')

In [18]:
# sve kolone koje su int ili float a imaju nan vrednosti popuni sa srednjom vrednosti te kolone
for column in clean_extended_data.select_dtypes(include=['int', 'float']).columns:
    clean_extended_data[column] = clean_extended_data[column].fillna(clean_extended_data[column].mean())
    
# Prikazi kolone sa nan vrednostima a da ih ima vise od 0 i sortiraj po broju nan vrednosti
nan_columns = clean_extended_data.columns[clean_extended_data.isna().sum() > 0]
nan_columns_counts = clean_extended_data[nan_columns].isna().sum().sort_values(ascending=False)
nan_columns_counts


Series([], dtype: float64)

In [19]:
# Prikazi ociscen dataset
clean_extended_data.head()


,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,HardHit,HardHit%,Events,CStr%,CSW%,L-WAR,competitive_runs,bolts,hp_to_1b,sprint_speed
0,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,217,0.500,434,0.220,0.270,10.2,237.0,6.0,4.18,28.1
1,10155,2018,Mike Trout,LAA,26,140,471,608,147,80,...,162,0.460,352,0.201,0.261,9.5,185.0,36.0,4.25,29.2
2,11579,2015,Bryce Harper,WSN,22,153,521,654,172,91,...,188,0.477,394,0.118,0.226,9.3,223.0,1.0,4.34,27.7
3,10155,2015,Mike Trout,LAA,23,159,575,682,172,93,...,205,0.486,422,0.207,0.282,9.3,236.0,73.0,4.20,29.6
4,15640,2017,Aaron Judge,NYY,25,155,542,678,154,75,...,186,0.550,338,0.157,0.290,8.7,209.0,0.0,4.50,28.0


In [20]:
# proveri da li ima nan vrednosti
clean_extended_data.isna().sum().sum()

0

In [21]:
# snimi dataset u novi csv fajl
clean_extended_data.to_csv('../resources/clean_extended_data.csv', index=False)